In [2]:
!pip install tez
!pip install torch==1.7.1
!pip install transformers==3.5.1

     |████████████████████████████████| 776.8MB 16kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101


In [3]:
import pandas as pd
import tez
import torch
import torch.nn as nn
import transformers
from sklearn import metrics, model_selection, preprocessing
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.utils import shuffle
import torch.nn.functional as nnf

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Work/Dan/BERT Model 2')

Mounted at /content/drive


In [6]:
class BERTBaseUncased(tez.Model):
    def __init__(self, num_train_steps, num_classes):
        super().__init__()
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-large-uncased", do_lower_case=True
        )
        self.bert = transformers.BertModel.from_pretrained("bert-large-uncased")
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(1024, num_classes)
        self.num_train_steps = num_train_steps
        self.step_scheduler_after = "batch"
    def fetch_optimizer(self):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.bias"]
        optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.001,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        opt = AdamW(optimizer_parameters, lr=3e-5)
        return opt
    def fetch_scheduler(self):
        sch = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=0, num_training_steps=self.num_train_steps
        )
        return sch
    def loss(self, outputs, targets):
        if targets is None:
            return None
        return nn.CrossEntropyLoss()(outputs, targets)
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    def forward(self, ids, mask, token_type_ids, targets=None):
        _, o_2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        b_o = self.bert_drop(o_2)
        output = self.out(b_o)
        loss = self.loss(output, targets)
        acc = self.monitor_metrics(output, targets)
        return output, loss, acc

In [8]:
import pickle
with open('lmap.pkl','rb') as f:
  d=pickle.load(f)

In [9]:
import transformers
DEVICE = "cuda"
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
BERT_PATH = "bert-large-uncased"
MODEL_PATH = "model.bin"
TRAINING_FILE = "train_data.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
# In[23]:

In [11]:
import time
import torch.nn as nn
import pandas as pd
MODEL = BERTBaseUncased(10000,len(d))

MODEL.load_state_dict(torch.load("model.bin"),strict=False)
MODEL.to(DEVICE)
MODEL.eval()

BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [12]:
tokenizer = TOKENIZER
max_len = MAX_LEN
sent_csv=pd.read_csv("test_data.csv")
sentence=sent_csv.utterance.values

In [17]:
maxindx=[]
predictions=[]
for index,value in enumerate(sentence):
    review = str(value)
    review = " ".join(review.split())
    inputs = tokenizer.encode_plus(
            review, None, add_special_tokens=True, max_length=max_len
        )
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]
    padding_length = max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)
    ids = ids.to(DEVICE, dtype=torch.long)
    token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    mask = mask.to(DEVICE, dtype=torch.long)
    outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)
    maxidx=torch.argmax(outputs[0],keepdim=False)
    maxindx.append(int(maxidx))
    print(d[int(maxidx)])  
    predictions.append(d[int(maxidx)])

leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave_guidelines
leave.leave

In [18]:
sent_csv["pred_idx"]=pd.DataFrame(maxindx)
sent_csv["pred_values"]=pd.DataFrame(predictions)
sent_csv.to_csv("output.csv")


In [19]:
sent_csv

,utterance,pred_idx,pred_values
0,staff overseas rate,150,leave.leave_guidelines
1,overseas fund transfer staff,150,leave.leave_guidelines
2,carry leave,150,leave.leave_guidelines
3,db share,150,leave.leave_guidelines
4,mobility,150,leave.leave_guidelines
...,...,...,...
3790,enroll share,150,leave.leave_guidelines
3791,change contribution share,150,leave.leave_guidelines
3792,reactivated,150,leave.leave_guidelines
3793,hi access denied due password expired,150,leave.leave_guidelines
